In [1]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
news_df = pd.read_csv('train.csv', nrows=3000)

In [3]:
pip install chardet

In [4]:
import chardet

with open('train.csv', 'rb') as f:
    result = chardet.detect(f.read(200000))  # Read first 200KB
print(result)

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


In [5]:
news_df.head()

,Unnamed: 0,title,text,label
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1
1,1,NaN,Did they post their votes for Hillary already?,1
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1


In [6]:
news_df.shape

(3000, 4)

In [7]:
news_df.isna().sum()

Unnamed: 0     0
title         23
text           1
label          0
dtype: int64

In [8]:
news_df = news_df.fillna(' ')

In [9]:
news_df.isna().sum()

Unnamed: 0    0
title         0
text          0
label         0
dtype: int64

In [10]:
news_df['content'] = news_df['title']

In [11]:
news_df

,Unnamed: 0,title,text,label,content
0,0,LAW ENFORCEMENT ON HIGH ALERT Following Threat...,No comment is expected from Barack Obama Membe...,1,LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1,1,,Did they post their votes for Hillary already?,1,
2,2,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...,"Now, most of the demonstrators gathered last ...",1,UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3,3,"Bobby Jindal, raised Hindu, uses story of Chri...",A dozen politically active pastors came here f...,0,"Bobby Jindal, raised Hindu, uses story of Chri..."
4,4,SATAN 2: Russia unvelis an image of its terrif...,"The RS-28 Sarmat missile, dubbed Satan 2, will...",1,SATAN 2: Russia unvelis an image of its terrif...
...,...,...,...,...,...
2995,2995,HOW HILLARY DESTROYED This Man’s Life To Hide ...,Does anyone even care that this American man w...,1,HOW HILLARY DESTROYED This Man’s Life To Hide ...
2996,2996,Republicans gear up for Supreme Court battle a...,WASHINGTON (Reuters) - Republican lawmakers an...,0,Republicans gear up for Supreme Court battle a...
2997,2997,Why’d You Do That? Printing Donald Trump’s Vul...,Coverage of the leaked 2005 audio tape of Don...,0,Why’d You Do That? Printing Donald Trump’s Vul...
2998,2998,Trump says didn't tell Flynn to discuss sancti...,WASHINGTON (Reuters) - President Donald Trump ...,0,Trump says didn't tell Flynn to discuss sancti...


In [12]:
news_df['content']

0       LAW ENFORCEMENT ON HIGH ALERT Following Threat...
1                                                        
2       UNBELIEVABLE! OBAMA’S ATTORNEY GENERAL SAYS MO...
3       Bobby Jindal, raised Hindu, uses story of Chri...
4       SATAN 2: Russia unvelis an image of its terrif...
                              ...                        
2995    HOW HILLARY DESTROYED This Man’s Life To Hide ...
2996    Republicans gear up for Supreme Court battle a...
2997    Why’d You Do That? Printing Donald Trump’s Vul...
2998    Trump says didn't tell Flynn to discuss sancti...
2999    Bruising debate brings a reluctant Trump aroun...
Name: content, Length: 3000, dtype: object

In [13]:
# stemming
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [14]:
news_df['content'] = news_df['content'].apply(stemming) 

In [15]:
news_df['content']

0       law enforc high alert follow threat cop white ...
1                                                        
2       unbeliev obama attorney gener say charlott rio...
3       bobbi jindal rais hindu use stori christian co...
4       satan russia unv imag terrifi new supernuk wes...
                              ...                        
2995      hillari destroy man life hide incompet benghazi
2996      republican gear suprem court battl scalia death
2997              print donald trump vulgar new york time
2998    trump say tell flynn discuss sanction russia w...
2999    bruis debat bring reluct trump around afghanis...
Name: content, Length: 3000, dtype: object

In [16]:
X = news_df['content'].values
y = news_df['label'].values

In [17]:
print(X)

['law enforc high alert follow threat cop white blacklivesmatt fyf terrorist video'
 ''
 'unbeliev obama attorney gener say charlott rioter peac protest home state north carolina video'
 ... 'print donald trump vulgar new york time'
 'trump say tell flynn discuss sanction russia would'
 'bruis debat bring reluct trump around afghanistan war']


In [18]:
vector = TfidfVectorizer()
vector.fit(X)
X = vector.transform(X)

In [19]:
print(X)

  (0, 120)	0.3455615443805985
  (0, 490)	0.33704443213108315
  (0, 1075)	0.26678178055236856
  (0, 1604)	0.3131813231600438
  (0, 1897)	0.30872892864241513
  (0, 1990)	0.38836586853071775
  (0, 2288)	0.2857229957314485
  (0, 2826)	0.2461416587102448
  (0, 4963)	0.2603281172508895
  (0, 4994)	0.2685448118439821
  (0, 5312)	0.14168086062411728
  (0, 5429)	0.2222785497392055
  (2, 297)	0.3011484453176523
  (2, 744)	0.3240946992467278
  (2, 819)	0.3423081656375447
  (2, 2031)	0.2723164103290891
  (2, 2323)	0.27001231886790056
  (2, 3419)	0.2356275529848124
  (2, 3447)	0.17441228850470553
  (2, 3653)	0.28603413424477786
  (2, 3892)	0.22709080665381978
  (2, 4210)	0.3423081656375447
  (2, 4329)	0.1631232606660867
  (2, 4737)	0.2029511635755935
  (2, 5178)	0.3423081656375447
  :	:
  (2996, 4332)	0.4570402570676526
  (2996, 4854)	0.3505990374124837
  (2997, 1453)	0.32781906124928545
  (2997, 3372)	0.2285267242910502
  (2997, 3843)	0.5862909298059142
  (2997, 5017)	0.24437959105761645
  (2997, 

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify=y, random_state=1)

In [21]:
X_train.shape

(2400, 5563)

In [22]:
X_test.shape

(600, 5563)

In [23]:
model = LogisticRegression()
model.fit(X_train,y_train)

LogisticRegression()

In [24]:
train_y_pred = model.predict(X_train)
print("train accurracy :",accuracy_score(train_y_pred,y_train))

train accurracy : 0.96


In [25]:
test_y_pred = model.predict(X_test)
print("train accurracy :",accuracy_score(test_y_pred,y_test))

train accurracy : 0.855


In [26]:
# prediction system

input_data = X_test[10]
prediction = model.predict(input_data)
if prediction[0] == 1:
    print('Fake news')
else:
    print('Real news')

Fake news


In [27]:
news_df['content'][10]

'gop senat smack punchabl alt right nazi internet'